# Getting Started with XPCS Toolkit

This tutorial introduces the basic concepts and usage of the XPCS Toolkit for analyzing X-ray Photon Correlation Spectroscopy (XPCS) and Small-Angle X-ray Scattering (SAXS) data.

## Overview

The XPCS Toolkit provides comprehensive analysis capabilities for:
- **XPCS analysis**: Multi-tau correlation functions, dynamics extraction
- **SAXS analysis**: 1D/2D scattering patterns, structure characterization
- **Stability analysis**: Time-resolved monitoring, radiation damage assessment
- **Data visualization**: Publication-quality plots and interactive analysis

## Learning Objectives

By the end of this tutorial, you will:
1. Understand the basic structure of XPCS data files
2. Load and inspect XPCS/SAXS data
3. Create basic visualizations
4. Navigate the toolkit's modular architecture
5. Access help and documentation


## Installation and Setup

First, let's ensure the toolkit is properly installed and import the necessary modules:

In [ ]:
# Standard scientific computing libraries

import matplotlib.pyplot as plt
import numpy as np

# XPCS Toolkit core components

# Configure matplotlib for inline plotting
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 12

print("XPCS Toolkit successfully imported!")
print("Available analysis modules: g2mod, saxs1d, saxs2d, stability")

## Understanding XPCS Data Files

The XPCS Toolkit works with HDF5-based data files that contain:
- **Detector images**: Raw scattering patterns
- **Correlation functions**: Pre-computed g₂(q,τ) data
- **SAXS profiles**: Azimuthally averaged intensities I(q)
- **Metadata**: Experimental parameters and geometry

### Creating a Mock Data File

For demonstration purposes, let's create some synthetic XPCS/SAXS data:

In [ ]:
# Create synthetic XPCS correlation data
def create_mock_xpcs_data():
    """Generate synthetic XPCS correlation function data"""
    # Time delays (logarithmic spacing)
    tau = np.logspace(-5, -1, 50)  # 10 μs to 0.1 s

    # Q-values
    q = np.linspace(0.01, 0.1, 10)  # Å⁻¹

    # Generate correlation functions with different relaxation rates
    g2 = np.zeros((len(tau), len(q)))
    g2_err = np.zeros((len(tau), len(q)))

    for i, q_val in enumerate(q):
        # Diffusion coefficient: D = 1e-12 m²/s
        D = 1e-12  # m²/s
        q_si = q_val * 1e10  # Convert to m⁻¹
        gamma = D * q_si**2  # Relaxation rate

        # Single exponential correlation function
        beta = 0.8  # Coherence factor
        g2[:, i] = 1.0 + beta * np.exp(-2 * gamma * tau)

        # Add realistic noise
        noise_level = 0.01
        g2[:, i] += noise_level * np.random.normal(0, 1, len(tau))
        g2_err[:, i] = noise_level * np.ones(len(tau))

    # Generate labels
    labels = [f"q={q_val:.3f} Å⁻¹" for q_val in q]

    return q, tau, g2, g2_err, labels


# Generate mock data
q_vals, tau_vals, g2_data, g2_errors, q_labels = create_mock_xpcs_data()

print("Generated XPCS data:")
print(f"  Q-range: {q_vals.min():.3f} to {q_vals.max():.3f} Å⁻¹")
print(f"  Time range: {tau_vals.min():.2e} to {tau_vals.max():.2e} s")
print(f"  Correlation matrix shape: {g2_data.shape}")

### Creating Mock SAXS Data

In [ ]:
def create_mock_saxs_data():
    """Generate synthetic SAXS scattering data"""
    # Q-values for SAXS
    q = np.logspace(-2, 0, 100)  # 0.01 to 1.0 Å⁻¹

    # Simulate spherical particles (radius = 50 Å)
    R = 50  # Å
    qR = q * R

    # Sphere form factor
    F_sphere = 3 * (np.sin(qR) - qR * np.cos(qR)) / (qR**3)
    F_sphere[0] = 1.0  # Limit as qR -> 0

    # Intensity with Porod background
    I0 = 1000  # Forward scattering
    background = 0.1
    I_saxs = I0 * F_sphere**2 + background

    # Add realistic noise
    I_saxs += np.sqrt(I_saxs) * np.random.normal(0, 0.1, len(q))

    return q, I_saxs


# Generate mock SAXS data
q_saxs, I_saxs = create_mock_saxs_data()

print("Generated SAXS data:")
print(f"  Q-range: {q_saxs.min():.3f} to {q_saxs.max():.3f} Å⁻¹")
print(f"  Intensity range: {I_saxs.min():.2f} to {I_saxs.max():.2f}")

## Basic Data Visualization

### XPCS Correlation Functions

Let's visualize the correlation function data using matplotlib:

In [ ]:
# Plot correlation functions for selected q-values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Left panel: Linear scale
for i in [0, 3, 6, 9]:  # Select representative q-values
    ax1.errorbar(
        tau_vals,
        g2_data[:, i],
        yerr=g2_errors[:, i],
        label=q_labels[i],
        marker="o",
        markersize=4,
        alpha=0.8,
    )

ax1.set_xlabel("Delay time τ (s)")
ax1.set_ylabel("g₂(q,τ)")
ax1.set_title("XPCS Correlation Functions")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right panel: Log-log scale
for i in [0, 3, 6, 9]:
    ax2.errorbar(
        tau_vals,
        g2_data[:, i],
        yerr=g2_errors[:, i],
        label=q_labels[i],
        marker="o",
        markersize=4,
        alpha=0.8,
    )

ax2.set_xscale("log")
ax2.set_yscale("log")
ax2.set_xlabel("Delay time τ (s)")
ax2.set_ylabel("g₂(q,τ)")
ax2.set_title("XPCS Correlation Functions (Log-Log)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("- Higher q-values show faster relaxation (steeper decay)")
print("- All curves approach g₂ = 1 at long times")
print("- Log-log plot reveals exponential relaxation behavior")

### SAXS Scattering Profile

In [ ]:
# Plot SAXS intensity profile
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Left panel: Log-log plot (standard SAXS representation)
ax1.plot(q_saxs, I_saxs, "b-", linewidth=2, label="Spherical particles (R=50Å)")
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.set_xlabel("q (Å⁻¹)")
ax1.set_ylabel("Intensity I(q)")
ax1.set_title("SAXS Profile (Log-Log)")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right panel: Kratky plot (q²I(q) vs q)
kratky_y = q_saxs**2 * I_saxs
ax2.plot(q_saxs, kratky_y, "r-", linewidth=2, label="Kratky Plot")
ax2.set_xlabel("q (Å⁻¹)")
ax2.set_ylabel("q²I(q)")
ax2.set_title("Kratky Plot")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSAXS analysis insights:")
print("- Form factor oscillations indicate monodisperse spheres")
print("- First minimum around q ≈ 0.15 Å⁻¹ corresponds to particle size")
print("- Kratky plot shows characteristic shape for globular particles")

## Basic Analysis with XPCS Toolkit

### Extracting Physical Parameters

Let's extract diffusion coefficients from the correlation data:

In [ ]:
def fit_single_exponential(tau, g2, g2_err=None):
    """Fit single exponential to correlation function"""
    from scipy.optimize import curve_fit

    def correlation_func(t, beta, gamma, baseline):
        return baseline + beta * np.exp(-2 * gamma * t)

    # Initial parameter guess
    p0 = [0.8, 1000, 1.0]  # beta, gamma, baseline

    # Fit with error weighting if available
    sigma = g2_err if g2_err is not None else None

    try:
        popt, pcov = curve_fit(correlation_func, tau, g2, p0=p0, sigma=sigma)
        perr = np.sqrt(np.diag(pcov))

        return {
            "beta": popt[0],
            "gamma": popt[1],
            "baseline": popt[2],
            "beta_err": perr[0],
            "gamma_err": perr[1],
            "baseline_err": perr[2],
            "fit_y": correlation_func(tau, *popt),
        }
    except:
        return None


# Fit correlation functions and extract relaxation rates
fit_results = []
diffusion_coeffs = []
diffusion_errors = []

for i, q_val in enumerate(q_vals):
    result = fit_single_exponential(tau_vals, g2_data[:, i], g2_errors[:, i])
    if result is not None:
        fit_results.append(result)

        # Calculate diffusion coefficient: D = Γ/q²
        gamma = result["gamma"]  # s⁻¹
        gamma_err = result["gamma_err"]
        q_si = q_val * 1e10  # Convert to m⁻¹

        D = gamma / (q_si**2)  # m²/s
        D_err = gamma_err / (q_si**2)

        diffusion_coeffs.append(D)
        diffusion_errors.append(D_err)
    else:
        fit_results.append(None)
        diffusion_coeffs.append(np.nan)
        diffusion_errors.append(np.nan)

# Convert to numpy arrays
diffusion_coeffs = np.array(diffusion_coeffs)
diffusion_errors = np.array(diffusion_errors)

print("Extracted diffusion coefficients:")
for i, (q_val, D, D_err) in enumerate(zip(q_vals, diffusion_coeffs, diffusion_errors)):
    if not np.isnan(D):
        print(f"  q = {q_val:.3f} Å⁻¹: D = ({D:.2e} ± {D_err:.2e}) m²/s")

### Visualizing Fit Results

In [ ]:
# Plot fits for selected q-values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Left panel: Correlation functions with fits
colors = plt.cm.viridis(np.linspace(0, 1, len(q_vals)))

for i, (q_val, color) in enumerate(zip(q_vals[::2], colors[::2])):
    # Plot data
    ax1.errorbar(
        tau_vals,
        g2_data[:, i * 2],
        yerr=g2_errors[:, i * 2],
        color=color,
        marker="o",
        markersize=3,
        alpha=0.7,
        label=f"q={q_val:.3f} Å⁻¹",
    )

    # Plot fit if available
    if fit_results[i * 2] is not None:
        ax1.plot(
            tau_vals, fit_results[i * 2]["fit_y"], color=color, linewidth=2, alpha=0.8
        )

ax1.set_xscale("log")
ax1.set_xlabel("Delay time τ (s)")
ax1.set_ylabel("g₂(q,τ)")
ax1.set_title("XPCS Data with Exponential Fits")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right panel: Diffusion coefficient vs q²
valid_mask = ~np.isnan(diffusion_coeffs)
if np.any(valid_mask):
    q_squared = (q_vals * 1e10) ** 2  # m⁻²

    ax2.errorbar(
        q_squared[valid_mask],
        diffusion_coeffs[valid_mask],
        yerr=diffusion_errors[valid_mask],
        marker="o",
        markersize=6,
        capsize=3,
        color="red",
        linewidth=2,
    )

    # Linear fit to check D = constant (expected for Brownian motion)
    mean_D = np.mean(diffusion_coeffs[valid_mask])
    ax2.axhline(
        mean_D,
        color="blue",
        linestyle="--",
        alpha=0.7,
        label=f"Mean D = {mean_D:.2e} m²/s",
    )

    ax2.set_xlabel("q² (m⁻²)")
    ax2.set_ylabel("Diffusion Coefficient (m²/s)")
    ax2.set_title("q-independence Check")
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.ticklabel_format(style="scientific", axis="both", scilimits=(0, 0))

plt.tight_layout()
plt.show()

# Calculate particle size using Stokes-Einstein relation
if np.any(valid_mask):
    kT = 4.1e-21  # J (room temperature)
    eta = 1e-3  # Pa·s (water viscosity)
    mean_D = np.mean(diffusion_coeffs[valid_mask])

    R_h = kT / (6 * np.pi * eta * mean_D)
    print("\nStokes-Einstein analysis:")
    print(f"  Mean diffusion coefficient: {mean_D:.2e} m²/s")
    print(f"  Hydrodynamic radius: {R_h * 1e9:.1f} nm")

## Working with Real Data Files

In practice, you'll work with actual XPCS data files. Here's how to load and analyze real data:

In [ ]:
# Example of loading real XPCS data (commented out - requires actual files)

"""
# Load XPCS data file
data_file = XpcsDataFile('path/to/your/xpcs_data.h5')

# Check what analysis types are available
print(f"Analysis type: {data_file.atype}")
print(f"Available datasets: {list(data_file.keys())}")

# Extract correlation function data
if 'Multitau' in data_file.atype:
    q, tau, g2, g2_err, labels = data_file.get_g2_data(
        q_range=(0.01, 0.1),  # Select q-range
        t_range=(1e-5, 1e-1)  # Select time range
    )

# Extract SAXS data
if 'Saxs1d' in data_file.atype:
    q_saxs, I_saxs, xlabel, ylabel = data_file.get_saxs_1d_data(
        q_range=(0.005, 0.5),
        norm_method='q2'  # Kratky normalization
    )

# Use analysis kernel for advanced processing
kernel = AnalysisKernel(data_file)
kernel.run_analysis(analysis_type='correlation')
"""

print("Real data loading example (commented out - requires actual data files)")
print("\nTo work with your own data:")
print("1. Use XpcsDataFile('your_file.h5') to load data")
print("2. Check available analysis types with .atype attribute")
print("3. Extract specific data using .get_g2_data() or .get_saxs_1d_data()")
print("4. Use analysis modules (g2mod, saxs1d, etc.) for visualization")

## Command Line Interface

The XPCS Toolkit also provides command-line tools for batch processing:

In [ ]:
# Display CLI help information
from IPython.display import Markdown

cli_help = """
### Available CLI Commands

```bash
# Plot correlation functions
xpcs-toolkit plot-g2 data.h5 --q-range 0.01 0.1 --output correlation_plot.png

# Analyze SAXS data
xpcs-toolkit plot-saxs-1d data.h5 --log-scale --kratky --output saxs_plot.png

# Convert file formats
xpcs-toolkit convert data.h5 --format nexus --output converted_data.nxs

# Batch processing
xpcs-toolkit batch-process *.h5 --analysis correlation --output-dir results/

# Quality assessment
xpcs-toolkit validate data.h5 --check-all --report quality_report.txt
```

### Getting Help

```bash
# General help
xpcs-toolkit --help

# Command-specific help
xpcs-toolkit plot-g2 --help
```
"""

display(Markdown(cli_help))

## Next Steps

This tutorial covered the basics of the XPCS Toolkit. To continue learning:

### Recommended Tutorials
1. **02_correlation_analysis.ipynb** - Advanced XPCS correlation analysis
2. **03_saxs_structure_analysis.ipynb** - Detailed SAXS structure characterization  
3. **04_stability_assessment.ipynb** - Time-resolved stability analysis
4. **05_advanced_fitting.ipynb** - Custom model fitting and parameter extraction

### Documentation Resources
- **API_REFERENCE.md** - Complete function documentation
- **FILE_FORMAT_GUIDE.md** - Data file format specifications
- **SCIENTIFIC_BACKGROUND.md** - Theoretical foundations
- **CLI_REFERENCE.md** - Command-line interface guide

### Getting Help
- Check module docstrings: `help(g2mod.pg_plot)`
- Use IPython's `?` operator: `XpcsDataFile?`
- Visit the online documentation
- Contact the development team for support

### Practice Exercises
1. Try loading different types of XPCS/SAXS data files
2. Experiment with different q-ranges and time ranges
3. Compare results from different fitting models
4. Create publication-quality figures with custom styling


---

*This tutorial is part of the XPCS Toolkit documentation. For the latest version and additional resources, visit the project repository.*